In [1]:
import datetime
import pandas as pd
import numpy
import pickle

In [2]:
matches_epl = pd.read_csv('Matches_EPL.csv')
players_df = pd.read_csv('data/player_attributes_transformed.csv')
matches_processed = pd.read_csv('Matches_EPL_processed.csv')

matches_epl = matches_epl.sort_values(by='date',ascending=False).reset_index(drop=False)

matches_processed['date'] = pd.to_datetime(matches_processed['date'], format="%Y-%m-%d")
matches_epl['date'] = pd.to_datetime(matches_epl['date'], format="%Y-%m-%d")

model = pickle.load(open('model_LR_epl.sav','rb'))
features = ['final_rating','home_last_five_score','away_last_five_score','home_opponent_score']


In [3]:
def get_last_five_score(team,date):
    streak_score=0
    last_five = matches_epl[((matches_epl['home_team_api_id']==team) | (matches_epl['away_team_api_id']==team)) & (matches_epl['date']<date)].head(5)
    #Last 5 not empty - i.e. no previous match data for this team id
    if not last_five.empty:
        #To determine whether the last match of team was at home or away to get the score difference 
        last_five['team_score'] = last_five.apply(lambda x: (x['home_team_goal']-x['away_team_goal'] if (x['home_team_api_id']==team) else x['away_team_goal']-x['home_team_goal'] ),axis=1)
        streak_score = last_five.team_score.sum()

        if type(streak_score)!= numpy.int64:
            return streak_score.values[0]
        else:
            return streak_score
    else:
        return 0

In [4]:
def get_last_five_opponent(team,date,opponent):
    last_five = matches_epl[((matches_epl['home_team_api_id']==team) & (matches_epl['away_team_api_id']==opponent)) & (matches_epl['date']<date)].head(5)
    if not last_five.empty:
        streak_score = last_five.groupby('id').apply(lambda x: x['home_team_goal']-x['away_team_goal']).sum()
        if type(streak_score)!= numpy.int64:
            return streak_score.values[0]
        else:
            return streak_score
    else:        
        return 0

In [ ]:
def get_result(result,home_id,away_id):
    if result==2:
        #Win
        return teams[teams['team_api_id']==home_id]['team_long_name'].values+" Wins Against " + teams[teams['team_api_id']==away_id]['team_long_name'].values
    elif result==1:
        return "Match Draw :" + teams[teams['team_api_id']==home_id]['team_long_name'].values +" vs "+ teams[teams['team_api_id']==away_id]['team_long_name'].values
    else:
        return teams[teams['team_api_id']==away_id]['team_long_name'].values+" Wins Against " + teams[teams['team_api_id']==home_id]['team_long_name'].values

In [ ]:
def create_features(home_team_api_id,away_team_api_id):
    processed_df = pd.DataFrame(columns=['final_rating','home_last_five_score','away_last_five_score','home_opponent_score'])
    date = datetime.datetime.today()
    home_last_match = matches_processed[(matches_processed['home_team_api_id']==home_team_api_id)  & (matches_processed['date']<date)]['home_rating'].head(1)
    away_last_match = matches_processed[(matches_processed['home_team_api_id']==away_team_api_id) & (matches_processed['date']<date)]['home_rating'].head(1)
    
    processed_df['final_rating'] = home_last_match.values - away_last_match.values
    processed_df['home_last_five_score'] = get_last_five_score(home_team_api_id,date)
    processed_df['away_last_five_score'] = get_last_five_score(away_team_api_id,date)
    processed_df['home_opponent_score'] = get_last_five_opponent(home_team_api_id,date,away_team_api_id)

    return processed_df

In [ ]:
teams = pd.read_csv('Team.csv')


In [ ]:
from flask import Flask, render_template, request
app = Flask(__name__)


@app.route('/')
def hello():
    return render_template('index.html')

@app.route('/index.html', methods = ['GET','POST'])
def testing():

    home_team_api_id = int(request.form['home'])
    away_team_api_id = int(request.form['away'])

    df = create_features(home_team_api_id, away_team_api_id)

    result = model.predict(df[features])
#     get_result(result,home_team_api_id,away_team_api_id)
        
    return render_template('index.html',valuexx=get_result(result,home_team_api_id,away_team_api_id)[0], home_id=home_team_api_id, away_id = away_team_api_id)
    
    
    
@app.route('/images/<cropzonekey>')
def images(cropzonekey):
    return render_template("images.html", title=cropzonekey)

@app.route('/fig/<cropzonekey>')
def fig(cropzonekey):
    fig = draw_polygons(cropzonekey)
    img = StringIO()
    fig.savefig(img)
    img.seek(0)
    return send_file(img, mimetype='image/png')

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Apr/2019 21:09:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:09:28] "GET /vendor/fontawesome-free/css/all.min.css HTTP/1.1" 404 -
127.0.0.1 - - [13/Apr/2019 21:09:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Apr/2019 21:12:13] "POST /index.html HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:12:13] "GET /vendor/fontawesome-free/css/all.min.css HTTP/1.1" 404 -
127.0.0.1 - - [13/Apr/2019 21:14:32] "GET /static/img/8197.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Apr/2019 21:14:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2019 21:14:40] "GET /vendor/fontawesome-free/css/all.min.css HTTP/1.1" 404 -
